# MiniRAG を postgres で実行する

- 精度は安定していないところもあるし、まだバグが潜んでいる可能性もあるが、一旦このレベルで OK

In [1]:
# 必要なライブラリのインポート
import os
import tempfile
from minirag import MiniRAG, QueryParam
from minirag.llm.hf import (
    hf_model_complete,
    hf_embed,
)
# from minirag.llm.openai import openrouter_openai_complete
from minirag.llm.openai import openai_complete_if_cache
from minirag.utils import EmbeddingFunc
from minirag.utils import (
    wrap_embedding_func_with_attrs,
    locate_json_string_body_from_string,
    safe_unicode_decode,
    logger,
)

import asyncpg
from psycopg_pool import AsyncConnectionPool
from minirag.kg.postgres_impl import PostgreSQLDB
from minirag.kg.postgres_impl import PGKVStorage
from minirag.kg.postgres_impl import PGVectorStorage
from minirag.kg.postgres_impl import PGGraphStorage
from minirag.kg.postgres_impl import PGDocStatusStorage

from transformers import AutoModel, AutoTokenizer
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception
from openai import RateLimitError, APIStatusError # openaiライブラリが投げる例外をインポート
import asyncio
import warnings
warnings.filterwarnings('ignore')

### データベース接続テスト

In [2]:
import psycopg
from psycopg.rows import dict_row

def get_conn():
    url = os.getenv("DATABASE_URL")
    if url:
        return psycopg.connect(url, row_factory=dict_row)
    # fallback: assemble from separate vars
    dsn = (
        f"host={os.getenv('PGHOST','db')} "
        f"port={os.getenv('PGPORT','5432')} "
        f"dbname={os.getenv('PGDATABASE','postgres')} "
        f"user={os.getenv('POSTGRES_USER','postgres')} "
        f"password={os.getenv('POSTGRES_PASSWORD')}"
    )
    return psycopg.connect(dsn, row_factory=dict_row)

with get_conn() as conn, conn.cursor() as cur:
    cur.execute("SELECT version();")
    print(cur.fetchone())

{'version': 'PostgreSQL 16.9 (Debian 16.9-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit'}


### ハイブリッドクエリのテスト
- 前提: 先に init_db.sh が実行されテーブルが作成されていること

In [3]:
def get_conn():
    """
    環境変数からデータベース接続情報を取得し、接続オブジェクトを返す。
    DATABASE_URLが設定されていればそれを使用し、なければ個別の変数を組み立てる。
    結果は辞書形式で返されるように設定済み。
    """
    url = os.getenv("DATABASE_URL")
    if url:
        # DATABASE_URLが設定されている場合
        return psycopg.connect(url, row_factory=dict_row)
    
    # DATABASE_URLがない場合、個別の環境変数からDSNを組み立てる
    # PGPASSWORDが設定されていないと接続に失敗するため、チェックを追加するとより親切
    password = os.getenv('PGPASSWORD')
    if not password:
        raise ValueError("環境変数 PGPASSWORD が設定されていません。")
        
    dsn = (
        f"host={os.getenv('PGHOST', 'localhost')} "
        f"port={os.getenv('PGPORT', '5432')} "
        f"dbname={os.getenv('PGDATABASE', 'postgres')} "
        f"user={os.getenv('POSTGRES_USER', 'postgres')} "
        f"password={password}"
    )
    return psycopg.connect(dsn, row_factory=dict_row)

def find_recommended_products_for_alice(_results):
    """
    指定されたSQLクエリを実行し、'Alice'が好む商品とベクトル的に類似した商品を取得する。
    """
    # 実行したいSQLクエリを三重クォートで定義
    # これにより、複数行のクエリを読みやすく記述できる
    sql_query = """
    SELECT
        p.id,
        p.name,
        p.embedding
    FROM
        public.products AS p
    JOIN
        -- cypher関数でグラフクエリを実行し、結果をテーブルのように扱う
        cypher('my_minirag_graph', $$
            MATCH (u:User {name: 'Alice'})-[:LIKES]->(prod:Product)
            RETURN prod.product_id
        $$) AS liked(product_id agtype)
    ON
        -- グラフクエリの結果(agtype)を整数にキャストしてproductsテーブルのIDと結合
        p.id = (liked.product_id)::INTEGER
    ORDER BY
        -- pgvectorの<->演算子で、指定ベクトルとのコサイン距離が近い順に並び替え
        p.embedding <=> '[0.1, 0.1, 0.2]';
    """

    try:
        # with文で接続とカーソルを管理し、処理終了後に自動でクローズする
        with get_conn() as conn, conn.cursor() as cur:
            print("データベースに接続し、クエリを実行します...")
            
            # クエリの実行
            cur.execute(sql_query)
            
            # 全ての実行結果を取得 (fetchall)
            # 1件だけなら fetchone(), 複数件なら fetchall() を使う
            results = cur.fetchall()
            
            print("\n--- クエリ実行結果 ---")
            if results:
                # 取得した結果を1行ずつ表示
                for row in results:
                    print(row)
                    _results.append(row)
            else:
                print("条件に一致する結果は見つかりませんでした。")
        return _results

    except psycopg.Error as e:
        print(f"データベースエラーが発生しました: {e}")
    except ValueError as e:
        print(f"設定エラーが発生しました: {e}")

query_results = []
query_results = find_recommended_products_for_alice(query_results)

データベースに接続し、クエリを実行します...

--- クエリ実行結果 ---
{'id': 1, 'name': '商品A', 'embedding': '[0.1,0.2,0.3]'}
{'id': 2, 'name': '商品B', 'embedding': '[0.2,0.1,0.9]'}


### マークダウンに整形

In [4]:
def convert_to_markdown(data: list[dict]) -> str:
    """
    辞書のリストをマークダウンのテーブル形式に変換します。

    Args:
        data: 辞書のリスト。各辞書がテーブルの1行に対応します。

    Returns:
        マークダウン形式のテーブル文字列。
    """
    if not data:
        return "データがありません。"

    # ヘッダーの作成 (最初のデータのキーから取得)
    headers = data[0].keys()
    header_line = "| " + " | ".join(headers) + " |"

    # 区切り線の作成
    separator_line = "| " + " | ".join(["---"] * len(headers)) + " |"

    # データ行の作成
    data_lines = []
    for row in data:
        # 各値を文字列に変換して結合
        values = [str(row.get(h, "")) for h in headers]
        data_lines.append("| " + " | ".join(values) + " |")

    # 全ての行を結合して返す
    return "\n".join([header_line, separator_line] + data_lines)

# --- 変換を実行して結果を表示 ---
markdown_table = convert_to_markdown(query_results)
print(markdown_table)

| id | name | embedding |
| --- | --- | --- |
| 1 | 商品A | [0.1,0.2,0.3] |
| 2 | 商品B | [0.2,0.1,0.9] |


In [5]:
#  id | name  |   embedding   
# ----+-------+---------------
#   1 | 商品A | [0.1,0.2,0.3]
#   2 | 商品B | [0.2,0.1,0.9]

# 上記の結果になればOK

In [6]:
# API KEY が見えないようにコメントアウト

# print(os.getenv("OPENROUTER_API_KEY"))
# print(os.getenv("OPENAI_API_KEY"))
print("Openrouter APIキーが設定されました")
print()

HF_TOKEN = os.getenv("HF_TOKEN")
# print(HF_TOKEN)
print("HuggingFace TOKEN が設定されました")

Openrouter APIキーが設定されました

HuggingFace TOKEN が設定されました


In [7]:
use_japanese_embedding = False

if use_japanese_embedding:
    # MINI モードでの回答ができなくなったので、逆に精度が落ちたかも。
    EMBEDDING_MODEL = "hotchpotch/static-embedding-japanese"
    EMBEDDING_DIM   = 1024
    TOKENIZER_MODEL = "hotchpotch/xlm-roberta-japanese-tokenizer"
    print("-------------- static-embedding-japanese を使用します --------------")
else:
    # 埋め込みモデルの設定
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    EMBEDDING_DIM   = 384
    print("-------------- all-MiniLM-L6-v2 を使用します --------------")


# LLMの設定
# LLM_MODEL = "Qwen/Qwen3-1.7B"  # または "Qwen/Qwen3-4B", "Qwen/Qwen3-1.7B" など
# LLM_MODEL = "jaeyong2/Qwen2.5-3B-Instruct-Ja-SFT"
# LLM_MODEL = "deepseek/deepseek-chat-v3-0324:free"
LLM_MODEL = "qwen/qwen3-235b-a22b-2507"



# 作業ディレクトリの作成
WORKING_DIR = "/tmp/minirag_demo"
os.makedirs(WORKING_DIR, exist_ok=True)

print(f"作業ディレクトリ: {WORKING_DIR}")


# DATA_PATH = args.datapath
# QUERY_PATH = args.querypath
# OUTPUT_PATH = args.outputpath
# print("USING LLM:", LLM_MODEL)
# print("USING WORKING DIR:", WORKING_DIR)

-------------- all-MiniLM-L6-v2 を使用します --------------
作業ディレクトリ: /tmp/minirag_demo


In [8]:
if use_japanese_embedding:

    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer(EMBEDDING_MODEL, device="cpu", token=HF_TOKEN)
    
    query = "美味しいラーメン屋に行きたい"
    docs = [
        "素敵なカフェが近所にあるよ。落ち着いた雰囲気でゆっくりできるし、窓際の席からは公園の景色も見えるんだ。",
        "新鮮な魚介を提供する店です。地元の漁師から直接仕入れているので鮮度は抜群ですし、料理人の腕も確かです。",
        "あそこは行きにくいけど、隠れた豚骨の名店だよ。スープが最高だし、麺の硬さも好み。",
        "おすすめの中華そばの店を教えてあげる。とりわけチャーシューが手作りで柔らかくてジューシーなんだ。",
    ]
    
    embeddings = model.encode([query] + docs)
    print(embeddings.shape)
    similarities = model.similarity(embeddings[0], embeddings[1:])
    for i, similarity in enumerate(similarities[0].tolist()):
        print(f"{similarity:.04f}: {docs[i]}")

else:
    
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL, token=HF_TOKEN)
    model = AutoModel.from_pretrained(EMBEDDING_MODEL, token=HF_TOKEN)


## transformer の API で使えるように変換する

In [9]:
if use_japanese_embedding:
    
    import torch
    from torch import nn
    from transformers import PreTrainedModel, PretrainedConfig
    from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions
    
    
    class StaticEmbeddingConfig(PretrainedConfig):
        model_type = "static-embedding"
    
        def __init__(self, vocab_size=32768, hidden_size=1024, pad_token_id=0, **kwargs):
            super().__init__(pad_token_id=pad_token_id, **kwargs)
            self.vocab_size = vocab_size
            self.hidden_size = hidden_size
    
    
    class StaticEmbeddingModel(PreTrainedModel):
        config_class = StaticEmbeddingConfig
    
        def __init__(self, config: StaticEmbeddingConfig):
            super().__init__(config)
            # ★ EmbeddingBag そのものでも OK ですが、
            #   シーケンス長をそろえて attention_mask で平均を取る方が扱いやすいので nn.Embedding に変更
            self.embedding = nn.Embedding(
                num_embeddings=config.vocab_size,
                embedding_dim=config.hidden_size,
                padding_idx=config.pad_token_id,
            )
            self.post_init()  # transformers の重み初期化
    
        def forward(self, input_ids, attention_mask=None, **kwargs):
            """
            - input_ids      : (batch, seq_len)
            - attention_mask : (batch, seq_len) — 0 は padding
            戻り値は Transformers 共通の BaseModelOutputWithPoolingAndCrossAttentions
            """
            if attention_mask is None:
                attention_mask = (input_ids != self.config.pad_token_id).int()
    
            token_embs = self.embedding(input_ids)                       # (B, L, H)
            # マスク付き平均プール
            masked_embs = token_embs * attention_mask.unsqueeze(-1)      # (B, L, H)
            lengths = attention_mask.sum(dim=1, keepdim=True).clamp(min=1e-8)  # (B, 1)
            sentence_emb = masked_embs.sum(dim=1) / lengths              # (B, H)
    
            return BaseModelOutputWithPoolingAndCrossAttentions(
                last_hidden_state=token_embs,  # ここでは token レベルをそのまま
                pooler_output=sentence_emb,    # 文ベクトル
                attentions=None,
                cross_attentions=None,
            )

In [10]:
if use_japanese_embedding:
    """
    SentenceTransformer 版 (hotchpotch/static-embedding-japanese) から
    StaticEmbeddingModel へ重みをコピーして保存するスクリプト
    """
    
    SRC = "hotchpotch/static-embedding-japanese"   # オリジナル
    DST = "./static-embedding-transformers"        # 保存先
    
    # ① SentenceTransformer を読み込む
    st_model = SentenceTransformer(SRC)
    embedding_weight = st_model[0].embedding.weight.data   # nn.EmbeddingBag の重みを取得
    
    # ② Config → Model を作成
    config = StaticEmbeddingConfig(
        vocab_size=embedding_weight.size(0),
        hidden_size=embedding_weight.size(1),
        pad_token_id=0,           # トークナイザの <pad> が id=0
    )
    model = StaticEmbeddingModel(config)
    
    # ③ 重みコピー
    with torch.no_grad():
        model.embedding.weight.copy_(embedding_weight)
    
    # ④ save_pretrained で書き出し
    model.save_pretrained(DST)
    # st_model.tokenizer.save_pretrained(DST)   # tokenizer.json なども一緒に保存
    
    print(f"✅ 変換完了 — 保存先: {DST}")

In [11]:
if use_japanese_embedding:
    
    MODEL_DIR = "./static-embedding-transformers"
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_MODEL)
    model     = StaticEmbeddingModel.from_pretrained(MODEL_DIR)
    
    sentences = [
        "美味しいラーメン屋に行きたい",
        "あそこは行きにくいけど、隠れた豚骨の名店だよ。スープが最高だし、麺の硬さも好み。",
    ]
    
    inputs = tokenizer(
        sentences,
        return_tensors="pt",
        padding=True,
        truncation=True,
        add_special_tokens=False,   # 元モデルは special tokens なし
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
        vecs = outputs.pooler_output     # (batch, hidden_size)
    
    print("shape:", vecs.shape)          # torch.Size([2, 1024])
    similarity = torch.nn.functional.cosine_similarity(vecs[0], vecs[1], dim=0)
    print("cosine:", similarity.item())

In [12]:
# どのエラーでリトライするかを定義
# 429 (RateLimitError) や サーバー側のエラー(5xx) でリトライするのが一般的
def should_retry(e: Exception) -> bool:
    if isinstance(e, RateLimitError):
        print(f"RateLimitError発生。リトライします...: {e}")
        return True
    # 5xx系のサーバーエラーでもリトライすることが多い
    if isinstance(e, APIStatusError) and e.status_code >= 500:
        print(f"サーバーエラー( {e.status_code} )発生。リトライします...: {e}")
        return True
    return False

@retry(
    wait=wait_random_exponential(multiplier=30, max=100), # 30, 32, 34, 38秒...とランダムな時間を加えて待つ（最大100秒）
    stop=stop_after_attempt(5), # 最大5回リトライする
    retry=retry_if_exception(should_retry) # 上で定義した条件の例外が発生した場合にリトライ
)
async def openrouter_openai_complete(
    prompt,
    system_prompt=None,
    history_messages=[],
    keyword_extraction=False,
    api_key: str = None,
    **kwargs,
) -> str:
    # if api_key:
    #     os.environ["OPENROUTER_API_KEY"] = api_key

    keyword_extraction = kwargs.pop("keyword_extraction", None)
    result = await openai_complete_if_cache(
        LLM_MODEL,  # change accordingly
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        base_url="https://openrouter.ai/api/v1",
        api_key=api_key,
        **kwargs,
    )
    if keyword_extraction:  # TODO: use JSON API
        return locate_json_string_body_from_string(result)
    return result

In [13]:
!pwd

!ls

/app
MiniRAG_on_postgres.ipynb  minirag  setup.py


### RAGシステムセットアップ

In [14]:
# テーブルチェックで以下のような表示大量に出るのは正常に動いている証拠
# ---
# PostgreSQL database error: relation "lightrag_doc_full" does not exist
# Failed to check table LIGHTRAG_DOC_FULL in PostgreSQL database
# ---
# SELECT 1 FROM LIGHTRAG_DOC_FULL LIMIT 1
# None
# ---

async def setup_rag_system():
    """RAGシステムを初期化し、準備ができたインスタンスを返す"""

    # 1. データベース設定
    db_config={
        "host": "postgres",    # これは Docker のサービス名「postgres」で指定
        "port": 5432,
        "user": os.getenv("POSTGRES_USER"),
        "password": os.getenv("POSTGRES_PASSWORD"),
        "database": os.getenv("POSTGRES_DB"),
    }

    # 2. PostgreSQLDBインスタンスを作成
    db_postgre = PostgreSQLDB(config=db_config)

    # 3. データベース接続を初期化
    print("------------------------- ポスグレに接続しています -------------------------")
    await db_postgre.initdb()
    print("------------------------- ポスグレに接続しました！ -------------------------")

    # 必要なテーブルが存在するかチェックし、なければ作成する
    print("------------------------- テーブルの存在を確認・作成しています -------------------------")
    await db_postgre.check_tables()
    print("------------------------- テーブルの準備が完了しました！ -------------------------")

    # 5. MiniRAGインスタンスを作成
    os.environ["AGE_GRAPH_NAME"] = "my_minirag_graph" # init_db.sh で指定したもの
    rag = MiniRAG(
        working_dir=WORKING_DIR,
        # llm_model_func=hf_model_complete,
        # llm_model_func=gemini_2_5_flash_complete,
        llm_model_func=openrouter_openai_complete,
        llm_model_max_token_size=200,
        llm_model_name=LLM_MODEL,
        embedding_func=EmbeddingFunc(
            embedding_dim=EMBEDDING_DIM,
            max_token_size=1000,
            func=lambda texts: hf_embed(
                texts,
                tokenizer=tokenizer,
                embed_model=model
            ),
        ),
        kv_storage="PGKVStorage",
        vector_storage="PGVectorStorage",
        graph_storage="PGGraphStorage",
        doc_status_storage="PGDocStatusStorage",
        vector_db_storage_cls_kwargs={
            "cosine_better_than_threshold": float(os.getenv("COSINE_THRESHOLD"))
        }
    )
    # データベースの情報を渡す
    rag.set_storage_client(db_postgre)    
    return rag

# RAGシステムをセットアップ
try:
    rag = await setup_rag_system()
    print("------------------------- MiniRAGが初期化されました！ -------------------------")
except Exception as e:
    print(f"RAGシステムのセットアップに失敗しました: {e}")

------------------------- ポスグレに接続しています -------------------------
------------------------- ポスグレに接続しました！ -------------------------
------------------------- テーブルの存在を確認・作成しています -------------------------
------------------------- テーブルの準備が完了しました！ -------------------------
------------------------- MiniRAGが初期化されました！ -------------------------


In [19]:
# 約12分かかった
import time
start_time = time.time()

# サンプルテキストデータ
sample_texts = [
    """
今日は素晴らしい一日でした。朝早く起きて、近所の公園を散歩しました。
桜の花が満開で、とても美しかったです。午後は友人と映画を見に行きました。
「君の名は。」という映画で、とても感動的でした。
夜は家族と一緒に夕食を取り、楽しい時間を過ごしました。
""",
    """
昨日は仕事で大きなプロジェクトが完了しました。
チーム全員で3ヶ月間取り組んできたAIシステムの開発が終わりました。
機械学習モデルの精度が95%を超え、クライアントからも高い評価をいただきました。
今夜はチームメンバーと祝賀会を開く予定です。
""",
    """
週末は料理に挑戦しました。初めてパスタを一から作ってみました。
小麦粉から麺を作るのは思っていたより難しかったですが、
最終的にはとても美味しいカルボナーラができました。
次回はリゾットに挑戦してみたいと思います。
""",
    """
読書が趣味で、最近は村上春樹の「ノルウェイの森」を読んでいます。
主人公の心情描写がとても繊細で、引き込まれます。
また、技術書も読んでおり、「深層学習」について学んでいます。
理論と実践のバランスが取れた良い本だと思います。
"""
]

# データの挿入
print("データを挿入中...")

async def insert_texts(rag_instance, texts):
    for i, text in enumerate(texts):
        print(f"テキスト {i+1}/{len(texts)} を挿入中...")
        await rag_instance.ainsert(text.strip())   # overwrite は デフォルトは False 

    print("\nすべてのデータが挿入されました！")


# イベントループが既に実行中の場合
try:
    await insert_texts(rag, sample_texts)
except RuntimeError:
    # 新しいループで実行
    asyncio.run(insert_texts(rag, sample_texts))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"処理時間: {elapsed_time:.4f}秒")

データを挿入中...
テキスト 1/4 を挿入中...
🚀 AINSERT called with overwrite=False
📥 Input: 1 documents
📥 IDs: None
📥 Metadatas: None
⠏ Processed 9 chunks, 38 entities(duplicated), 19 relations(duplicated)
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_i

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  4.80batch/s]


KG successfully indexed.
テキスト 2/4 を挿入中...
🚀 AINSERT called with overwrite=False
📥 Input: 1 documents
📥 IDs: None
📥 Metadatas: None
⠏ Processed 9 chunks, 34 entities(duplicated), 29 relations(duplicated)
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デ

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.09batch/s]


KG successfully indexed.
テキスト 3/4 を挿入中...
🚀 AINSERT called with overwrite=False
📥 Input: 1 documents
📥 IDs: None
📥 Metadatas: None
⠏ Processed 9 chunks, 43 entities(duplicated), 34 relations(duplicated)
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デ

Generating embeddings: 100%|██████████| 2/2 [00:00<00:00,  3.14batch/s]


KG successfully indexed.
テキスト 4/4 を挿入中...
🚀 AINSERT called with overwrite=False
📥 Input: 1 documents
📥 IDs: None
📥 Metadatas: None
⠏ Processed 9 chunks, 39 entities(duplicated), 39 relations(duplicated)
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デ

Generating embeddings: 100%|██████████| 2/2 [00:00<00:00,  2.86batch/s]

KG successfully indexed.

すべてのデータが挿入されました！
処理時間: 86.8267秒


In [20]:
# 約3分かかった

def print_test_case(name: str, answer: str, source: str, ):
    """テストケースの結果を分かりやすく表示するヘルパー関数"""
    print(f"\n--- {name} ---")
    print("Answer:", answer)
    print("Source:", source)
    print("-" * (len(name) + 8))


# # サンプルクエリ
# queries = [
#     "映画について教えて",
#     "仕事のプロジェクトはどうでしたか？",
#     "料理で何を作りましたか？",
#     "読んでいる本について教えて",
#     "散歩について詳しく教えて"
# ]

# # 各モードでクエリを実行。この3つがある
# modes = ["naive", "mini", "light"]

# for query in queries:
#     print(f"\n{'='*50}")
#     print(f"クエリ: {query}")
#     print(f"{'='*50}")

#     for mode in modes:
#         print(f"\n--- {mode.upper()}モード ---")
#         try:
#             answer = rag.query(query, param=QueryParam(mode=mode))     # .replace("\n", "").replace("\r", "")
#             print(f"回答: {answer}")
#         except Exception as e:
#             print(f"エラー: {e}")


async def run_queries(rag_instance, queries):
    # 各モードでクエリを実行。この3つがある
    modes = ["naive", "mini", "light"]

    for query in queries:
        print(f"\n{'='*50}")
        print(f"クエリ: {query}")
        print(f"{'='*50}")

        for mode in modes:
            print(f"\n--- {mode.upper()}モード ---")
            try:
                # 非同期でクエリを実行
                _answer, _source = await rag_instance.aquery(query, param=QueryParam(mode=mode))
                print_test_case(mode+"モード", _answer, _source)
            except Exception as e:
                print(f"エラー: {e}")


start_time = time.time()

sample_queries = [
    "映画について教えて",
    "仕事のプロジェクトはどうでしたか？",
    "料理で何を作りましたか？",
    "読んでいる本について教えて",
    "散歩について詳しく教えて"
]

# イベントループが既に実行中の場合
try:
    await run_queries(rag, sample_queries)
except RuntimeError:
    # 新しいループで実行
    asyncio.run(run_queries(rag, sample_queries))


end_time = time.time()
elapsed_time = end_time - start_time
print(f"処理時間: {elapsed_time:.4f}秒")


クエリ: 映画について教えて

--- NAIVEモード ---
🎯 Using temporary distance threshold: -1.0 (original: 0.2)
🔍 No metadata filter applied
📊 Query returned 9 results
✅ Result 1: id=chunk-7da419c35b..., distance=0.5192127640726866
✅ Result 2: id=chunk-da7de75046..., distance=0.47385024694712685
🔎 Total records in LIGHTRAG_DOC_CHUNKS: 9
🔎 Records with metadata: 4
🔎 Raw metadata and distance values:
   - ID: chunk-7da419c35b...
     Raw metadata: {"city": "Tokyo", "year": 2024, "category": "weather"}
     Extracted category: weather
     Distance: 0.5192127640726866
     Metadata type: <class 'str'>
   - ID: chunk-da7de75046...
     Raw metadata: {"year": 2023, "country": "Japan", "category": "geography"}
     Extracted category: geography
     Distance: 0.47385024694712685
     Metadata type: <class 'str'>
   - ID: chunk-cf733b0e4f...
     Raw metadata: {}
     Extracted category: None
     Distance: 0.45656117074148617
     Metadata type: <class 'str'>

--- naiveモード ---
Answer: 日本では映画が広く親しまれており、国内外問わず多く

## ここからは metadata_filter と time_filter を使ったテスト

## 初回も2回目もうまくいった(2回目というのは以下のセルを2回実行するということ)
- 精度にブレがあり、回答できる時とできない時があるが、今回は精度は求めていないのでこれで OK とする

In [17]:
from datetime import datetime, timedelta
from typing import Optional, Dict, List, Any

async def run_tests():
    """RAGシステムのフィルタリング機能をテストするメイン関数"""
    # 0. RAGシステムのセットアップ
    try:
        rag_with_filter = await setup_rag_system()
        print("------------------------- RAGシステムが初期化されました！ -------------------------")
    except Exception as e:
        print(f"RAGシステムのセットアップに失敗しました: {e}")
        return

    # 1. テストデータの準備と登録
    # タイムスタンプのテストのため、登録を複数回に分ける
    print("\n[ステップ1] データの登録を開始します...")

    # データセット1
    docs1 = [
        {"doc_id": "doc1", "content": "今日は東京でとても良い天気です。", "metadata": {"category": "weather", "city": "Tokyo", "year": 2024}},
        {"doc_id": "doc2", "content": "昨日の大阪は雨でした。", "metadata": {"category": "weather", "city": "Osaka", "year": 2024}},
    ]
    await rag_with_filter.ainsert(
        input=[d["content"] for d in docs1],
        ids=[d["doc_id"] for d in docs1],
        metadatas=[d["metadata"] for d in docs1],
        overwrite=True
    )
    print("データセット1 (doc1, doc2) を登録しました。")
    
    time_after_docs1 = datetime.utcnow()  # → Postgres は UTC で解釈するため、必ず UTC にすること
    await asyncio.sleep(10)  # タイムスタンプを明確に区別するため10秒待機
    
    # データセット2
    docs2 = [
        {"doc_id": "doc3", "content": "日本の首都は東京です。最近は兵庫も主要な都市に入るか議論されています。", "metadata": {"category": "geography", "country": "Japan", "year": 2023}},
        {"doc_id": "doc4", "content": "大阪は日本の主要な都市の一つです。最近は秋田も主要な都市に入るか議論されています。", "metadata": {"category": "geography", "country": "Japan", "year": 2023}},
        {"doc_id": "doc5", "content": "What is the capital of Japan? It's Tokyo.", "metadata": {}},  # メタデータなし
    ]
    await rag_with_filter.ainsert(
        input=[d["content"] for d in docs2],
        ids=[d["doc_id"] for d in docs2],
        metadatas=[d["metadata"] for d in docs2],
        overwrite=True
    )
    print("データセット2 (doc3, doc4, doc5) を登録しました。")
    time_after_docs2 = datetime.utcnow()  # → Postgres は UTC で解釈するため、必ず UTC にすること

    print("\n[ステップ2] 検索フィルタリングのテストを実行します...")

    # 思考プロセスと回答言語の指示
    instruction = "英語で思考して日本語で回答してください。回答は情報源に基づいて回答し、回答できない場合は`回答できない`と答えてください。"

    # --- メタデータフィルタリングのテスト ---
    param1 = QueryParam(mode="light",
                        metadata_filter={"category": "weather"})
    query1 = f"今日の天気について教えてください。{instruction}"
    answer1, source1 = await rag_with_filter.aquery(query1, param=param1)
    print_test_case("Test Case 1: 単一メタデータでフィルタ ('weather')", answer1, source1)

    param2 = QueryParam(mode="light",
                        metadata_filter={"category": "geography", "country": "Japan"})
    query2 = f"日本の地理に関する情報を情報源に基づいて回答してください。{instruction}"
    answer2, source2 = await rag_with_filter.aquery(query2, param=param2)
    print_test_case("Test Case 2: 複数メタデータでフィルタ (AND条件)。ちゃんと成功したのでフィルター機能は正常かも。ただ、ちょっと精度が低い", answer2, source2,)

    param3 = QueryParam(mode="light",
                        metadata_filter={"city": "北海道"})
    query3 = f"北海道の天気はどうですか？{instruction}"
    answer3, source3 = await rag_with_filter.aquery(query3, param=param3)
    print_test_case("Test Case 3: 存在しないメタデータ値でフィルタ（失敗例）", answer3, source3)

    query4 = f"東京について何か知っていますか？{instruction}"
    answer4, source4 = await rag_with_filter.aquery(query4) # フィルタなし
    print_test_case("Test Case 4: フィルタなしで検索", answer4, source4)

    param5 = QueryParam(mode="light",
                        metadata_filter={"category": "weather"})
    query5 = f"今日 の 東京 の 天気 について詳しく教えてください。{instruction}" # 'Tokyo'はdoc5にも含まれるが、こっちはフィルタで除外されるはず
    answer5, source5 = await rag_with_filter.aquery(query5, param=param5)
    print_test_case("Test Case 5: メタデータフィルタはメタデータなし文書を除外(正解: 天気について回答できる。これ精度が低い気がする)", answer5, source5)

    param6 = QueryParam(mode="light",
                        metadata_filter={"year": 2023})
    query6 = f"2023年の日本の出来事について何か知っていますか？{instruction}"
    answer6, source6 = await rag_with_filter.aquery(query6, param=param6)
    print_test_case("Test Case 6: 数値のメタデータでフィルタ", answer6, source6)

    # --- 時間フィルタリングのテスト ---
    param7 = QueryParam(mode="light",
                        start_time=time_after_docs1.isoformat())
    query7 = f"日本に関する最近登録された情報を教えてください。{instruction}"
    answer7, source7 = await rag_with_filter.aquery(query7, param=param7)
    print_test_case("Test Case 7: 'start_time'でフィルタ (後半の登録データのみ)。正解: 後半のデータのみで回答できること", answer7, source7)

    param8 = QueryParam(mode="light",
                        end_time=time_after_docs1.isoformat())
    query8 = f"大阪のことについて、なにか分かることを教えてください。{instruction}"
    answer8, source8 = await rag_with_filter.aquery(query8, param=param8)
    print_test_case("Test Case 8: 'end_time'でフィルタ (前半の登録データのみ)。正解: 大阪の天気について回答できること。これは精度が低いかも", answer8, source8)

    param9 = QueryParam(mode="light",
                        start_time=time_after_docs1.isoformat(), end_time=(time_after_docs1 - timedelta(seconds=10)).isoformat())
    query9 = f"東京について何か情報は登録されましたか？{instruction}"
    answer9, source9 = await rag_with_filter.aquery(query9, param=param9) # スタート時間よりも10秒前で指定している
    print_test_case("Test Case 9: 時間範囲指定 (結果なし)", answer9, source9)

    # --- 複合フィルタリングのテスト ---
    param10 = QueryParam(mode="light",
                         metadata_filter={"category": "geography"}, start_time=time_after_docs1.isoformat())
    query10 = f"日本の地理に関して、最近の話題を探しています。{instruction}"
    answer10, source10 = await rag_with_filter.aquery(query10, param=param10)
    print_test_case("Test Case 10: メタデータと時間の複合フィルタ (成功例。後半の情報を使う)", answer10, source10)


    param10 = QueryParam(mode="light",
                         metadata_filter={"category": "geography"}, end_time=time_after_docs1.isoformat())
    query10 = f"日本の地理に関して、最近の話題を探しています。{instruction}"
    answer10, source10 = await rag_with_filter.aquery(query10, param=param10)
    print_test_case("こっちはエンドタイムでやってみた。(失敗例。metadata_filter: geography とタイムフィルター)", answer10, source10)


    param10 = QueryParam(mode="light",
                                                                     end_time=time_after_docs1.isoformat())
    query10 = f"日本の地理に関して、最近の話題を探しています。{instruction}"
    answer10, source10 = await rag_with_filter.aquery(query10, param=param10)
    print_test_case("こっちはエンドタイムでやってみた。(失敗例。タイムフィルターのみ)", answer10, source10)


    
    param11 = QueryParam(mode="light",
                         metadata_filter={"category": "weather"}, start_time=time_after_docs1.isoformat())
    query11 = f"最近の天気予報について教えてください。{instruction}"
    answer11, source11 = await rag_with_filter.aquery(query11, param=param11)
    print_test_case("Test Case 11: メタデータと時間の複合フィルタ (失敗例)", answer11, source11)

    # --- その他のテスト ---
    query12 = f"フランスの首都について教えてください。{instruction}"
    answer12, source12 = await rag_with_filter.aquery(query12)
    print_test_case("Test Case 12: クエリがどの文書にもマッチしない", answer12, source12)



await run_tests()

------------------------- ポスグレに接続しています -------------------------
------------------------- ポスグレに接続しました！ -------------------------
------------------------- テーブルの存在を確認・作成しています -------------------------
------------------------- テーブルの準備が完了しました！ -------------------------
------------------------- RAGシステムが初期化されました！ -------------------------

[ステップ1] データの登録を開始します...
🚀 AINSERT called with overwrite=True
📥 Input: 2 documents
📥 IDs: ['doc1', 'doc2']
📥 Metadatas: [{'category': 'weather', 'city': 'Tokyo', 'year': 2024}, {'category': 'weather', 'city': 'Osaka', 'year': 2024}]
🔥 OVERWRITE MODE: Deleting existing chunks for 2 documents
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exist

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00, 72.82batch/s]


💾 Upserting chunk 'chunk-7da419c35b...' with metadata: {"city": "Tokyo", "year": 2024, "category": "weather"}
🗑️  Deleted chunks vector records for doc_ids: ['doc2', 'doc1']
🗑️  Deleted text_chunks records for doc_ids: ['doc2', 'doc1']
⚙️  Processing doc 'doc2', status_doc.metadata = {"city": "Osaka", "year": 2024, "category": "weather"}
📦 Created 1 chunks for doc 'doc2'
   └─ Chunk 'chunk-4b17ba18b5...' metadata: {"city": "Osaka", "year": 2024, "category": "weather"}


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00, 75.84batch/s]

💾 Upserting chunk 'chunk-4b17ba18b5...' with metadata: {"city": "Osaka", "year": 2024, "category": "weather"}


...

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00, 67.80batch/s]

💾 Upserting chunk 'chunk-94cf9b5fb2...' with metadata: {}


⠏ Processed 9 chunks, 40 entities(duplicated), 31 relations(duplicated)
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): cr

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.86batch/s]


KG successfully indexed.
データセット2 (doc3, doc4, doc5) を登録しました。

[ステップ2] 検索フィルタリングのテストを実行します...
🎯 Using temporary distance threshold: -1.0 (original: 0.2)
📊 Query returned 60 results
✅ Result 1: id=Ename-c6eff27e5e..., distance=0.4268295231826875
✅ Result 2: id=Ename-b7ca548d57..., distance=0.42457626068349175
🔎 Total records in LIGHTRAG_VDB_ENTITY: 122
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postgres_impl.py): create_graph already exists
★デバッグ(postg

### メモ

- Mini モード／Light モードで使用している chunks_vdb.query() すべてに debug=False を明示したので、混乱を招くようなデバッグ表示は出ないようにした。ただし、Naive モードはデバッグ表示が ON になっているのでデバッグ表示の量がちょっと多い。また、Mini モードは一部 ベクトル検索を使っている関係で Naive モードと同じデバッグ表示が表示される。
- デバッグ表示が ON になると以下が表示される。これらの表示が Mini モードや Naive モードで表示されるということ
  - 「Total records in LIGHTRAG_DOC_CHUNKS: X」
  - 「Raw metadata and distance values: ~~~」
    - 「Raw metadata and distance values:」はトップ3が表示される仕様
- ドキュメントIDが重複すると Deleted が出るのは仕様でOK。一回消して再インデックスを作成しているから。
- 大量に出ている表示は正常
  - 「★デバッグ(postgres_impl.py): create_graph already exists」がたくさん出るのは正常。ナレッジグラフがあるかどうか必ずチェックする仕様になっているため。
  - Mini モードの「SELECT * FROM cypher('～～～」 や None も正常
- 次元が混在するとエラーになるので、embedding model は基本的に統一した方が良い
- doc_id はなくてもいいけど、管理しやすくなるのであった方が良さそう

In [18]:
# 以下が色々出るのは仕様なので正常な動作。いっぱい出るのは並列処理しているらしい
# 正常に動いていることを確認したいので、あえて出したままにしている。
# ---------------------------------------------------------
# ★デバッグ(postgres_impl.py): create_graph already exists



# postgres16_age_pgvector_container  | 2025-07-22 04:43:01.467 UTC [1672] ERROR:  graph "my_minirag_graph" already exists
# postgres16_age_pgvector_container  | 2025-07-22 04:43:01.467 UTC [1672] STATEMENT:  select create_graph('my_minirag_graph')



# SELECT * FROM cypher('my_minirag_graph', $$
#                      MATCH path = (start:Entity {node_id: "xe980b1e69cab"})-[*1..2]-(neighbor:Entity)
#                      RETURN [n in nodes(path) | properties(n).node_id] AS path_nodes,
#                             [r in relationships(path) | r] AS path_edges
#                    $$) AS (path_nodes agtype, path_edges agtype)
# None

# 上記のクエリも合っているし Miniモード も正常に機能しているので問題ない。多分表示している箇所は PostgreSQLDB クラスの execute メソッドだと思う。




# INFO:minirag:Using the label default for PostgreSQL as identifier
# INFO:minirag:Connected to PostgreSQL database at postgres:5432/my_database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_doc_full" does not exist
# ERROR:minirag:Failed to check table LIGHTRAG_DOC_FULL in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_doc_full" does not exist
# INFO:minirag:Created table LIGHTRAG_DOC_FULL in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_doc_chunks" does not exist
# ERROR:minirag:Failed to check table LIGHTRAG_DOC_CHUNKS in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_doc_chunks" does not exist
# INFO:minirag:Created table LIGHTRAG_DOC_CHUNKS in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_vdb_entity" does not exist
# ERROR:minirag:Failed to check table LIGHTRAG_VDB_ENTITY in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_vdb_entity" does not exist
# INFO:minirag:Created table LIGHTRAG_VDB_ENTITY in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_vdb_relation" does not exist
# ERROR:minirag:Failed to check table LIGHTRAG_VDB_RELATION in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_vdb_relation" does not exist
# INFO:minirag:Created table LIGHTRAG_VDB_RELATION in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_llm_cache" does not exist
# ERROR:minirag:Failed to check table LIGHTRAG_LLM_CACHE in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_llm_cache" does not exist
# INFO:minirag:Created table LIGHTRAG_LLM_CACHE in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_doc_status" does not exist
# ERROR:minirag:Failed to check table LIGHTRAG_DOC_STATUS in PostgreSQL database
# ERROR:minirag:PostgreSQL database error: relation "lightrag_doc_status" does not exist
# INFO:minirag:Created table LIGHTRAG_DOC_STATUS in PostgreSQL database
# INFO:minirag:Finished checking all tables in PostgreSQL database
# INFO:minirag:Logger initialized for working directory: /tmp/minirag_demo

# 上記の表示されている ERROR ログは、一見すると問題のように見えますが、実際には 「テーブルが存在しないことを検知した」 という正常な動作の一部です。check_tables メソッドは、このエラーを意図的に発生させて、テーブルが存在しない場合にのみ作成処理を行うように設計されています。